<a href="https://colab.research.google.com/github/zhpinkman/chatbot-using-seq2seq/blob/master/ChatBot_With_Seq2Seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [67]:
import numpy as np
import tensorflow as tf
import pickle
from tensorflow.keras import layers , activations , models , preprocessing
from tensorflow.keras import preprocessing , utils
import os
import yaml
from gensim.models import Word2Vec
import re

In [68]:
!wget https://github.com/shubham0204/Dataset_Archives/blob/master/chatbot_nlp.zip?raw=true -O chatbot_nlp.zip
!unzip chatbot_nlp.zip

--2020-09-13 15:31:17--  https://github.com/shubham0204/Dataset_Archives/blob/master/chatbot_nlp.zip?raw=true
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/shubham0204/Dataset_Archives/raw/master/chatbot_nlp.zip [following]
--2020-09-13 15:31:18--  https://github.com/shubham0204/Dataset_Archives/raw/master/chatbot_nlp.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/shubham0204/Dataset_Archives/master/chatbot_nlp.zip [following]
--2020-09-13 15:31:18--  https://raw.githubusercontent.com/shubham0204/Dataset_Archives/master/chatbot_nlp.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443..

In [69]:
dir_path = 'chatbot_nlp/data'
files_list = os.listdir(dir_path + os.sep)

questions = list()
answers = list()

for filepath in files_list:
    stream = open( dir_path + os.sep + filepath , 'rb')
    docs = yaml.safe_load(stream)
    conversations = docs['conversations']
    for con in conversations:
        if len( con ) > 2 :
            questions.append(con[0])
            replies = con[ 1 : ]
            ans = ''
            for rep in replies:
                ans += ' ' + rep
            answers.append( ans )
        elif len( con )> 1:
            questions.append(con[0])
            answers.append(con[1])

answers_with_tags = list()
for i in range( len( answers ) ):
    if type( answers[i] ) == str:
        answers_with_tags.append( answers[i] )
    else:
        questions.pop( i )

answers = list()
for i in range( len( answers_with_tags ) ) :
    answers.append( '<START> ' + answers_with_tags[i] + ' <END>' )

In [70]:
questions[:5], answers[:5]

(['you sound like hal',
  'you sound like yoda',
  'have you seen blade runner',
  'xfind spiderman',
  'when did teknolust'],
 ["<START> to me that's a great compliment. <END>",
  '<START> my grammatical patterns are sufficient for me to understand you. <END>',
  '<START> sure i have seen <END>',
  '<START> what is spiderman. <END>',
  '<START> teknolust was released in 2002. <END>'])

In [71]:
len(questions)

564

In [72]:
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts( questions + answers )
VOCAB_SIZE = len(tokenizer.word_index) + 1
print( 'VOCAB SIZE : {}'.format( VOCAB_SIZE ))

VOCAB SIZE : 1894


In [73]:
vocab = []
for word in tokenizer.word_index:
    vocab.append( word )

In [74]:
vocab[:5]

['end', 'start', 'you', 'a', 'i']

In [75]:
# encoder_input_data
tokenized_questions = tokenizer.texts_to_sequences(questions)
maxlen_question = max([ len(x) for x in tokenized_questions ])
encoder_input_data = preprocessing.sequence.pad_sequences(tokenized_questions , maxlen=maxlen_question , padding='post')
print(encoder_input_data.shape , maxlen_question)

(564, 22) 22


In [76]:
# decoder_input_data
tokenized_answers = tokenizer.texts_to_sequences(answers)
maxlen_answers = max([ len(x) for x in tokenized_answers ])
decoder_input_data = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post')
print(decoder_input_data.shape , maxlen_answers)

(564, 74) 74


In [77]:
# decoder_output_data
tokenized_answers = tokenizer.texts_to_sequences(answers)
for i in range(len(tokenized_answers)) :
    tokenized_answers[i] = tokenized_answers[i][1:]
padded_answers = preprocessing.sequence.pad_sequences(tokenized_answers , maxlen=maxlen_answers , padding='post')
decoded_output_data = utils.to_categorical(padded_answers , VOCAB_SIZE)
print(decoder_output_data.shape)

(564, 74, 1894)


In [78]:
encoder_inputs = tf.keras.layers.Input(shape=(maxlen_question, ))
encoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True )(encoder_inputs)
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM( 200 , return_state=True)(encoder_embedding)
encoder_states = [ state_h , state_c ]

In [79]:
decoder_inputs = tf.keras.layers.Input(shape=(maxlen_answers,  ))
decoder_embedding = tf.keras.layers.Embedding(VOCAB_SIZE, 200 , mask_zero=True)(decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM(200 , return_state=True , return_sequences=True)
decoder_outputs , _ , _ = decoder_lstm(decoder_embedding , initial_state=encoder_states)
decoder_dense = tf.keras.layers.Dense(VOCAB_SIZE , activation=tf.keras.activations.softmax) 
output = decoder_dense(decoder_outputs)

In [80]:
model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output)
model.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "functional_33"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_17 (InputLayer)           [(None, 22)]         0                                            
__________________________________________________________________________________________________
input_18 (InputLayer)           [(None, 74)]         0                                            
__________________________________________________________________________________________________
embedding_6 (Embedding)         (None, 22, 200)      378800      input_17[0][0]                   
__________________________________________________________________________________________________
embedding_7 (Embedding)         (None, 74, 200)      378800      input_18[0][0]                   
______________________________________________________________________________________

In [81]:
history = model.fit([encoder_input_data , decoder_input_data], decoder_output_data, batch_size=50, epochs=150 ) 
model.save('model.h5') 

Epoch 1/150
12/12 [==============================] - 7s 621ms/step - loss: 1.4521 - accuracy: 0.0773
Epoch 2/150
12/12 [==============================] - 7s 613ms/step - loss: 1.2923 - accuracy: 0.0708
Epoch 3/150
12/12 [==============================] - 7s 615ms/step - loss: 1.1347 - accuracy: 0.0757
Epoch 4/150
12/12 [==============================] - 7s 611ms/step - loss: 1.1093 - accuracy: 0.0890
Epoch 5/150
12/12 [==============================] - 7s 615ms/step - loss: 1.0969 - accuracy: 0.1055
Epoch 6/150
12/12 [==============================] - 7s 624ms/step - loss: 1.0853 - accuracy: 0.1468
Epoch 7/150
12/12 [==============================] - 7s 619ms/step - loss: 1.0715 - accuracy: 0.1487
Epoch 8/150
12/12 [==============================] - 7s 607ms/step - loss: 1.0543 - accuracy: 0.1495
Epoch 9/150
12/12 [==============================] - 7s 624ms/step - loss: 1.0385 - accuracy: 0.1514
Epoch 10/150
12/12 [==============================] - 7s 623ms/step - loss: 1.0250 - accura

In [82]:
def make_inference_models():
    
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    
    decoder_state_input_h = tf.keras.layers.Input(shape=( 200 ,))
    decoder_state_input_c = tf.keras.layers.Input(shape=( 200 ,))
    
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    return encoder_model , decoder_model

In [83]:
def str_to_tokens( sentence : str ):
    words = sentence.lower().split()
    tokens_list = list()
    for word in words:
        tokens_list.append( tokenizer.word_index[word]) 
    return preprocessing.sequence.pad_sequences( [tokens_list] , maxlen=maxlen_question , padding='post')

In [84]:
enc_model , dec_model = make_inference_models()
for _ in range(10):
    states_values = enc_model.predict(str_to_tokens(input( 'Enter question : ' )))
    empty_target_seq = np.zeros((1, 1))
    empty_target_seq[0, 0] = tokenizer.word_index['start']
    stop_condition = False
    decoded_translation = ''
    while not stop_condition :
        dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )
        sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
        sampled_word = None
        for word , index in tokenizer.word_index.items() :
            if sampled_word_index == index :
                decoded_translation += ' {}'.format( word )
                sampled_word = word
        
        if sampled_word == 'end' or len(decoded_translation.split()) > maxlen_answers:
            stop_condition = True
            
        empty_target_seq = np.zeros( ( 1 , 1 ) )  
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        states_values = [ h , c ] 

    print( decoded_translation )

Enter question : you are cruel
 i couldn't have not an oxymoron end
Enter question : you are smart
 you are you for a lot of clones end
Enter question : you are kind
 i have a lot of things end
Enter question : what is basketball
 a game with tall players end
Enter question : what is barcelona
 a weird sci fi trilogy written by the direction of prograssion of a wave from the functions of a computer to put it simply the computer to put it simply the computer to put it simply the computer to put it simply the computer to put it simply the computer to put it simply the computer to put it simply the computer to put it simply the computer to put it simply an electronic component
Enter question : good morning
 robots rule end
Enter question : hi
 hello end
Enter question : what is your age
 i am still young by your standards end
Enter question : what can you do
 i am not a chat robot to run some friends in the computer to put it end
Enter question : do you drink
 my brain does not require an